In [1]:
!pip install gensim

In [2]:
#!pip install tensorly

#imported packages
import os
import numpy as np
import pandas as pd
import tensorly as tl
from tensorly.decomposition import tucker
import math
from scipy import stats
import csv
from matplotlib.pyplot import figure
import matplotlib
from sklearn.metrics import mean_squared_error as rmse
import time

# LDA for clusterring
import gensim
from gensim import corpora
import matplotlib.pyplot as plt

In [4]:
#defined functions
#data column methods

#convert data to array
def getseqid(targetval):
    #convert taz idx
    targetidx = {}
    idxtarget = {}
    idx = 0
    for i in targetval:
        targetidx[i] = idx        
        idxtarget[idx] = i
        idx += 1
    return(targetidx, idxtarget)

def data2arr(datapath, binaryidx=True, to2d=True):
    taz2idx = lambda x: tazidx[x]
    str2time = lambda x: int(int(x.split(':')[0])*12 + int(x.split(':')[1])/5)
    timelen = 24*12 #with 5-min interval
    newid2idx = lambda x: newididx[x]
    
    datapath = raw_dt_path[0]
    dt = pd.read_csv(datapath)
    
    #convert time and location
    tazidx, idxtaz = getseqid(np.unique(dt['taz']))
    dt['tazidx'] = dt['taz'].apply(taz2idx)
    dt['timeidx'] = dt['time'].apply(str2time)

    newididx, idxnewid = getseqid(np.unique(dt['newid']))
    dt['newididx'] = dt['newid'].apply(newid2idx)
    
    if to2d == True:
        dt['ltidx'] = dt['tazidx']*timelen+dt['timeidx']
        nrow = len(newididx)
        ncol = timelen * len(tazidx)

        arr = np.zeros((nrow, ncol))
        dt = dt[['newididx','ltidx', 'sum']].to_numpy()

        #assign binary index
        for i, j, count in dt:
            if binaryidx == True:
                arr[i, j] = 1
            else:
                arr[i, j] = count
        return(arr)
    else:
        nrow = len(newididx)
        ntaz = len(tazidx)
        ntime = timelen
        
        arr = np.zeros((nrow, ntaz, ntime))
        dt = dt[['newididx', 'tazidx', 'timeidx', 'sum']].to_numpy()
        
        for i, j, k, count in dt:
            if binaryidx == True:
                arr[i, j, k] = 1
            else:
                arr[i, j, k] = sum
        return(arr)
    

In [5]:
#process the fcd data
path = "G:/My Drive/2021/Bias/sumo_simulation/"
#os.listdir()

raw_dt_path = []
for i in os.listdir(path):
    if len(i) == 12 and '2017' in i:
        raw_dt_path.append(path+i)

In [15]:
#!pip install fastdtw
#clustering
import numpy as np
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

In [56]:
# Generate synthetic binary time series data
# Replace this with your actual data
# Calculate pairwise DTW distances
def get_dtw_dist(arr):
    num_samples = arr.shape[0]
    dtw_distances = np.zeros((num_samples, num_samples))
    #print(dtw_distances.shape)
    for i in range(num_samples):
        for j in range(i, num_samples):
            distance, _ = fastdtw(arr[i], arr[j], dist=euclidean)
            print(distance)
            dtw_distances[i, j] = distance
            dtw_distances[j, i] = distance
    return(dtw_distances)

def normalized_dtw(dtw_distances):
    # Normalize DTW distances (optional but can help with K-Means)
    max_distance = np.max(dtw_distances)
    normalized_dtw_distances = dtw_distances / max_distance
    return(normalized_dtw_distances)

def conduct_cluster(normalized_dtw_distances, num_clusters):
    # Apply K-Means clustering
    #num_clusters = 3  # Adjust as needed
    #normalized_dtw_distances = get_dtw_dist(arr, return_mx)
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    cluster_labels = kmeans.fit_predict(normalized_dtw_distances)
    # Print cluster labels
    #print("Cluster Labels:")
    #print(cluster_labels)
    db_index = davies_bouldin_score(normalized_dtw_distances, cluster_labels)
    return(db_index)

In [41]:
#check the daily pattern across all the taz
cluster_num = np.arange(2, 11)
arr_test = data2arr(raw_dt_path[0])

In [ ]:
dtw_distance = get_dtw_dist(arr_test)

0.0
39.0
12.0
4.0
3.0
5.0
14.0
24.0
19.0
4.0
24.0
3.0
23.0
12.0
67.0
11.0
4.0
55.0
7.0
4.0
4.0
3.0
5.0
3.0
5.0
11.0
13.0
5.0
5.0
14.0
26.0
16.0
3.0
43.0
0.0
12.0
10.0
5.0
19.0
7.0
7.0
12.0
6.0
29.0
31.0
13.0
23.0
3.0
95.0
74.0
3.0
73.0
5.0
5.0
3.0
9.0
71.0
2.0
5.0
25.0
20.0
16.0
9.0
16.0
34.0
5.0
7.0
9.0
4.0
21.0
9.0
8.0
23.0
16.0
31.0
8.0
6.0
17.0
7.0
27.0
48.0
8.0
6.0
24.0
4.0
3.0
4.0
25.0
5.0
4.0
2.0
3.0
16.0
37.0
4.0
2.0
41.0
45.0
4.0
13.0
26.0
4.0
32.0
9.0
6.0
4.0
16.0
3.0
3.0
93.0
5.0
4.0
3.0
2.0
20.0
4.0
2.0
42.0
7.0
3.0
7.0
61.0
9.0
2.0
3.0
2.0
3.0
6.0
122.0
2.0
46.0
21.0
14.0
6.0
14.0
51.0
3.0
137.0
14.0
7.0
4.0
6.0
6.0
7.0
38.0
2.0
52.0
36.0
3.0
4.0
5.0
39.0
6.0
5.0
84.0
5.0
25.0
3.0
57.0
35.0
2.0
3.0
26.0
2.0
6.0
5.0
5.0
12.0
12.0
4.0
23.0
18.0
10.0
3.0
10.0
9.0
8.0
6.0
6.0
5.0
4.0
6.0
6.0
5.0
2.0
3.0
6.0
5.0
7.0
6.0
40.0
5.0
4.0
3.0
4.0
20.0
29.0
6.0
6.0
3.0
7.0
5.0
5.0
4.0
3.0
37.0
54.0
4.0
19.0
9.0
14.0
5.0
10.0
5.0
6.0
4.0
4.0
36.0
5.0
2.0
5.0
19.0
9.0
44.0
28.0
20.0
5.0

6.0
2.0
50.0
2.0
6.0
4.0
9.0
16.0
6.0
5.0
3.0
6.0
12.0
3.0
5.0
24.0
7.0
37.0
10.0
5.0
2.0
10.0
17.0
4.0
6.0
31.0
27.0
3.0
26.0
3.0
4.0
3.0
3.0
3.0
3.0
9.0
9.0
12.0
9.0
7.0
1.0
2.0
7.0
77.0
2.0
14.0
4.0
46.0
6.0
4.0
6.0
17.0
3.0
47.0
40.0
25.0
3.0
17.0
30.0
3.0
8.0
57.0
62.0
6.0
34.0
5.0
42.0
6.0
2.0
3.0
6.0
18.0
2.0
10.0
5.0
8.0
14.0
2.0
37.0
55.0
5.0
33.0
51.0
3.0
61.0
9.0
35.0
34.0
9.0
17.0
5.0
3.0
4.0
42.0
3.0
8.0
15.0
6.0
10.0
28.0
23.0
28.0
10.0
24.0
26.0
5.0
48.0
4.0
3.0
25.0
45.0
107.0
5.0
7.0
30.0
54.0
6.0
36.0
10.0
51.0
5.0
18.0
2.0
5.0
12.0
103.0
1.0
6.0
1.0
16.0
5.0
3.0
102.0
2.0
2.0
6.0
23.0
32.0
6.0
27.0
3.0
58.0
3.0
3.0
6.0
16.0
94.0
99.0
26.0
11.0
2.0
7.0
14.0
45.0
58.0
5.0
16.0
32.0
3.0
29.0
5.0
64.0
32.0
3.0
6.0
14.0
23.0
5.0
7.0
6.0
118.0
5.0
5.0
1.0
73.0
25.0
28.0
12.0
3.0
20.0
16.0
3.0
1.0
12.0
2.0
6.0
7.0
13.0
5.0
1.0
30.0
6.0
13.0
6.0
27.0
3.0
44.0
3.0
4.0
5.0
6.0
11.0
15.0
2.0
6.0
2.0
44.0
7.0
2.0
5.0
39.0
35.0
3.0
10.0
14.0
37.0
3.0
3.0
8.0
54.0
29.0
11.0
8.0
31

13.0
4.0
5.0
93.0
10.0
12.0
13.0
31.0
3.0
7.0
6.0
13.0
5.0
2.0
34.0
29.0
20.0
8.0
26.0
79.0
3.0
3.0
5.0
59.0
3.0
5.0
4.0
6.0
0.0
6.0
6.0
1.0
5.0
7.0
92.0
5.0
88.0
3.0
28.0
5.0
8.0
2.0
1.0
4.0
6.0
6.0
5.0
43.0
0.0
6.0
5.0
13.0
3.0
24.0
6.0
1.0
7.0
3.0
13.0
7.0
28.0
6.0
5.0
5.0
1.0
35.0
44.0
19.0
18.0
4.0
3.0
3.0
14.0
13.0
29.0
12.0
7.0
3.0
5.0
4.0
9.0
6.0
3.0
3.0
3.0
6.0
6.0
3.0
60.0
15.0
62.0
25.0
5.0
82.0
57.0
71.0
9.0
3.0
6.0
5.0
5.0
9.0
7.0
4.0
11.0
5.0
3.0
14.0
27.0
2.0
20.0
5.0
4.0
3.0
6.0
3.0
11.0
16.0
5.0
6.0
3.0
3.0
8.0
42.0
5.0
2.0
82.0
15.0
99.0
7.0
50.0
5.0
6.0
2.0
37.0
52.0
49.0
3.0
13.0
6.0
13.0
5.0
9.0
24.0
5.0
24.0
4.0
19.0
36.0
26.0
5.0
3.0
83.0
10.0
3.0
16.0
14.0
7.0
2.0
3.0
3.0
22.0
2.0
5.0
7.0
10.0
4.0
3.0
4.0
31.0
17.0
3.0
99.0
4.0
11.0
6.0
5.0
2.0
60.0
7.0
7.0
3.0
13.0
61.0
23.0
16.0
9.0
47.0
44.0
7.0
10.0
6.0
9.0
8.0
5.0
11.0
56.0
5.0
17.0
3.0
11.0
3.0
5.0
14.0
3.0
1.0
1.0
2.0
29.0
14.0
5.0
1.0
17.0
36.0
7.0
18.0
6.0
5.0
5.0
7.0
3.0
4.0
110.0
3.0
23.0
5.0
1.0
4.0


11.0
4.0
8.0
4.0
5.0
5.0
4.0
20.0
7.0
17.0
9.0
3.0
2.0
23.0
4.0
13.0
6.0
9.0
5.0
6.0
26.0
9.0
78.0
10.0
3.0
9.0
5.0
12.0
14.0
6.0
11.0
6.0
3.0
5.0
5.0
36.0
39.0
5.0
10.0
9.0
2.0
6.0
36.0
6.0
10.0
3.0
30.0
3.0
6.0
5.0
5.0
18.0
3.0
20.0
73.0
5.0
25.0
42.0
7.0
36.0
15.0
52.0
2.0
25.0
9.0
6.0
28.0
85.0
3.0
2.0
5.0
5.0
19.0
5.0
3.0
16.0
6.0
60.0
15.0
5.0
3.0
4.0
5.0
10.0
60.0
3.0
6.0
23.0
3.0
3.0
9.0
55.0
45.0
18.0
12.0
13.0
5.0
3.0
4.0
10.0
3.0
3.0
41.0
5.0
11.0
6.0
16.0
25.0
5.0
3.0
63.0
3.0
1.0
1.0
4.0
5.0
42.0
5.0
52.0
12.0
7.0
9.0
5.0
5.0
16.0
18.0
3.0
5.0
5.0
38.0
20.0
2.0
6.0
6.0
5.0
6.0
5.0
8.0
16.0
12.0
15.0
3.0
103.0
2.0
3.0
52.0
33.0
128.0
24.0
77.0
4.0
7.0
4.0
3.0
38.0
2.0
42.0
12.0
55.0
6.0
18.0
3.0
20.0
4.0
4.0
3.0
4.0
6.0
16.0
4.0
18.0
7.0
29.0
10.0
3.0
3.0
5.0
3.0
3.0
17.0
38.0
8.0
169.0
34.0
3.0
33.0
12.0
25.0
3.0
5.0
10.0
9.0
10.0
5.0
30.0
12.0
4.0
5.0
13.0
86.0
23.0
3.0
5.0
0.0
41.0
3.0
16.0
26.0
4.0
3.0
6.0
18.0
8.0
5.0
10.0
3.0
5.0
22.0
6.0
9.0
50.0
22.0
3.0
5.0
179.0
2

3.0
69.0
3.0
4.0
3.0
25.0
14.0
28.0
5.0
11.0
13.0
5.0
7.0
21.0
2.0
5.0
10.0
5.0
16.0
33.0
6.0
6.0
41.0
5.0
12.0
14.0
10.0
18.0
25.0
13.0
46.0
24.0
5.0
8.0
5.0
6.0
11.0
119.0
3.0
3.0
22.0
6.0
6.0
58.0
5.0
13.0
6.0
32.0
5.0
58.0
10.0
4.0
2.0
3.0
9.0
3.0
57.0
13.0
12.0
11.0
16.0
8.0
8.0
4.0
7.0
4.0
4.0
14.0
3.0
10.0
3.0
3.0
3.0
6.0
9.0
10.0
17.0
6.0
5.0
3.0
7.0
8.0
5.0
4.0
30.0
5.0
5.0
8.0
2.0
2.0
31.0
61.0
80.0
7.0
3.0
21.0
3.0
14.0
27.0
6.0
19.0
5.0
11.0
10.0
5.0
44.0
16.0
9.0
4.0
19.0
7.0
26.0
15.0
56.0
3.0
11.0
3.0
4.0
15.0
47.0
3.0
3.0
24.0
6.0
13.0
33.0
10.0
56.0
16.0
2.0
15.0
7.0
3.0
35.0
37.0
3.0
12.0
22.0
6.0
27.0
101.0
2.0
10.0
18.0
21.0
5.0
10.0
24.0
12.0
7.0
3.0
6.0
4.0
56.0
6.0
20.0
19.0
5.0
2.0
2.0
9.0
6.0
13.0
11.0
8.0
19.0
5.0
35.0
5.0
5.0
6.0
4.0
7.0
5.0
3.0
7.0
5.0
2.0
3.0
34.0
3.0
9.0
61.0
1.0
3.0
5.0
2.0
15.0
3.0
3.0
71.0
16.0
8.0
23.0
3.0
11.0
11.0
5.0
3.0
5.0
4.0
158.0
4.0
13.0
2.0
16.0
24.0
23.0
14.0
4.0
16.0
59.0
7.0
16.0
5.0
3.0
3.0
2.0
4.0
8.0
5.0
28.0
4.0
2.0
14

...

20.0
43.0
24.0
16.0
10.0
12.0
30.0
15.0
18.0
13.0
76.0
18.0
18.0
50.0
28.0
16.0
19.0
13.0
14.0
20.0
50.0
15.0
16.0
18.0
83.0
15.0
31.0
12.0
49.0
19.0
18.0
16.0
19.0
29.0
24.0
33.0
12.0
33.0
15.0
60.0
45.0
16.0
16.0
15.0
16.0
16.0
18.0
22.0
17.0
14.0
18.0
18.0
31.0
16.0
31.0
15.0
16.0
16.0
13.0
45.0
20.0
16.0
45.0
10.0
51.0
43.0
17.0
16.0
18.0
20.0
15.0
16.0
16.0
14.0
27.0
16.0
16.0
69.0
25.0
18.0
15.0
15.0
38.0
13.0
11.0
82.0
19.0
30.0
24.0
13.0
56.0
51.0
21.0
26.0
33.0
16.0
52.0
18.0
15.0
18.0
13.0
16.0
150.0
21.0
16.0
18.0
15.0
47.0
19.0
17.0
15.0
16.0
19.0
19.0
19.0
16.0
12.0
15.0
14.0
32.0
16.0
16.0
16.0
33.0
56.0
13.0
16.0
16.0
14.0
13.0
19.0
75.0
14.0
31.0
14.0
16.0
14.0
20.0
14.0
13.0
16.0
21.0
16.0
15.0
32.0
24.0
25.0
16.0
16.0
25.0
18.0
32.0
10.0
15.0
16.0
18.0
18.0
18.0
70.0
18.0
27.0
18.0
15.0
26.0
63.0
17.0
14.0
17.0
14.0
15.0
32.0
13.0
18.0
16.0
127.0
35.0
29.0
14.0
20.0
19.0
39.0
33.0
23.0
14.0
19.0
27.0
22.0
16.0
75.0
15.0
17.0
19.0
17.0
25.0
16.0
18.0
34.0
18.0
12.0
44.

35.0
18.0
35.0
27.0
60.0
10.0
27.0
16.0
19.0
16.0
11.0
30.0
22.0
17.0
46.0
19.0
17.0
17.0
38.0
11.0
17.0
14.0
18.0
18.0
16.0
18.0
15.0
45.0
29.0
15.0
16.0
15.0
31.0
13.0
22.0
14.0
15.0
10.0
35.0
15.0
22.0
22.0
18.0
18.0
16.0
13.0
12.0
18.0
15.0
16.0
22.0
19.0
13.0
35.0
15.0
26.0
13.0
15.0
16.0
23.0
27.0
16.0
15.0
18.0
18.0
27.0
44.0
18.0
22.0
13.0
19.0
14.0
11.0
19.0
16.0
20.0
19.0
12.0
16.0
23.0
18.0
10.0
17.0
16.0
16.0
21.0
17.0
20.0
82.0
6.0
36.0
9.0
16.0
16.0
19.0
18.0
13.0
20.0
23.0
15.0
10.0
14.0
18.0
21.0
20.0
17.0
13.0
18.0
24.0
13.0
17.0
16.0
21.0
17.0
15.0
21.0
26.0
21.0
83.0
29.0
31.0
16.0
24.0
13.0
28.0
15.0
30.0
12.0
62.0
16.0
18.0
27.0
45.0
36.0
16.0
15.0
14.0
11.0
44.0
14.0
14.0
14.0
14.0
44.0
16.0
14.0
17.0
15.0
12.0
18.0
16.0
16.0
12.0
39.0
31.0
16.0
16.0
31.0
14.0
30.0
17.0
13.0
16.0
16.0
25.0
18.0
14.0
12.0
15.0
18.0
15.0
16.0
17.0
17.0
11.0
17.0
18.0
18.0
24.0
45.0
16.0
18.0
51.0
64.0
10.0
16.0
12.0
16.0
16.0
18.0
14.0
12.0
19.0
34.0
21.0
16.0
32.0
18.0
20.0
24.0
36

17.0
14.0
15.0
15.0
15.0
13.0
16.0
37.0
18.0
15.0
27.0
18.0
15.0
18.0
13.0
18.0
14.0
10.0
31.0
21.0
18.0
17.0
20.0
11.0
26.0
15.0
18.0
15.0
39.0
49.0
42.0
12.0
15.0
21.0
27.0
49.0
22.0
8.0
23.0
25.0
20.0
33.0
29.0
14.0
18.0
17.0
18.0
22.0
19.0
16.0
65.0
17.0
65.0
18.0
17.0
21.0
15.0
14.0
18.0
11.0
21.0
15.0
8.0
20.0
47.0
16.0
15.0
10.0
15.0
13.0
16.0
27.0
16.0
16.0
20.0
16.0
36.0
15.0
16.0
19.0
15.0
14.0
18.0
23.0
31.0
13.0
18.0
16.0
18.0
44.0
95.0
16.0
15.0
16.0
56.0
14.0
22.0
24.0
28.0
17.0
34.0
15.0
16.0
22.0
14.0
21.0
10.0
49.0
16.0
46.0
16.0
14.0
32.0
15.0
16.0
35.0
16.0
21.0
19.0
18.0
17.0
16.0
26.0
16.0
12.0
27.0
16.0
24.0
10.0
16.0
15.0
38.0
17.0
25.0
15.0
99.0
18.0
18.0
11.0
18.0
16.0
14.0
19.0
17.0
43.0
22.0
31.0
14.0
15.0
26.0
13.0
16.0
18.0
25.0
64.0
16.0
18.0
69.0
32.0
25.0
17.0
17.0
39.0
15.0
13.0
15.0
13.0
15.0
21.0
14.0
18.0
15.0
15.0
16.0
14.0
60.0
16.0
22.0
8.0
15.0
17.0
17.0
15.0
14.0
13.0
43.0
63.0
15.0
12.0
13.0
18.0
15.0
23.0
22.0
15.0
28.0
30.0
16.0
17.0
37.0
16.

46.0
10.0
40.0
18.0
33.0
12.0
53.0
28.0
31.0
16.0
15.0
10.0
13.0
15.0
24.0
16.0
14.0
26.0
15.0
13.0
58.0
23.0
73.0
21.0
13.0
22.0
16.0
12.0
16.0
47.0
88.0
11.0
15.0
20.0
64.0
54.0
14.0
39.0
26.0
16.0
14.0
45.0
10.0
15.0
31.0
14.0
20.0
16.0
26.0
113.0
103.0
15.0
15.0
15.0
21.0
18.0
19.0
16.0
18.0
18.0
12.0
17.0
16.0
21.0
18.0
16.0
14.0
16.0
13.0
12.0
18.0
24.0
20.0
17.0
23.0
17.0
22.0
43.0
14.0
43.0
20.0
15.0
14.0
50.0
100.0
13.0
23.0
10.0
12.0
32.0
16.0
20.0
13.0
26.0
14.0
14.0
18.0
16.0
89.0
13.0
16.0
27.0
19.0
19.0
15.0
14.0
16.0
25.0
17.0
27.0
96.0
22.0
28.0
39.0
14.0
8.0
30.0
24.0
16.0
17.0
13.0
52.0
20.0
17.0
62.0
38.0
16.0
15.0
26.0
15.0
19.0
33.0
23.0
19.0
24.0
12.0
14.0
17.0
23.0
12.0
17.0
16.0
12.0
14.0
15.0
13.0
11.0
11.0
10.0
16.0
15.0
20.0
28.0
16.0
12.0
13.0
102.0
14.0
41.0
22.0
18.0
16.0
6.0
31.0
42.0
20.0
20.0
31.0
15.0
18.0
13.0
14.0
18.0
8.0
12.0
16.0
29.0
18.0
17.0
16.0
19.0
15.0
15.0
18.0
17.0
18.0
15.0
8.0
89.0
18.0
16.0
25.0
16.0
17.0
27.0
41.0
16.0
18.0
16.0
17.0


26.0
10.0
21.0
14.0
29.0
15.0
16.0
12.0
30.0
14.0
18.0
16.0
15.0
8.0
12.0
16.0
18.0
19.0
15.0
19.0
20.0
15.0
15.0
12.0
16.0
33.0
16.0
17.0
17.0
16.0
13.0
16.0
19.0
15.0
23.0
20.0
13.0
13.0
12.0
16.0
28.0
17.0
21.0
18.0
23.0
11.0
42.0
38.0
16.0
36.0
13.0
14.0
82.0
48.0
18.0
17.0
15.0
18.0
20.0
51.0
16.0
14.0
27.0
15.0
17.0
23.0
33.0
14.0
31.0
15.0
17.0
16.0
64.0
21.0
27.0
30.0
17.0
16.0
42.0
13.0
17.0
16.0
36.0
39.0
12.0
14.0
16.0
25.0
20.0
16.0
37.0
14.0
60.0
34.0
9.0
19.0
39.0
16.0
24.0
23.0
62.0
18.0
19.0
14.0
15.0
16.0
19.0
16.0
38.0
15.0
20.0
15.0
12.0
69.0
17.0
56.0
18.0
65.0
17.0
30.0
22.0
14.0
33.0
16.0
16.0
12.0
18.0
16.0
16.0
24.0
32.0
19.0
14.0
12.0
18.0
16.0
15.0
10.0
16.0
24.0
14.0
15.0
22.0
34.0
16.0
14.0
56.0
13.0
17.0
16.0
12.0
17.0
16.0
28.0
13.0
12.0
14.0
100.0
23.0
21.0
16.0
14.0
15.0
15.0
15.0
16.0
11.0
48.0
13.0
16.0
18.0
20.0
39.0
18.0
11.0
20.0
16.0
12.0
15.0
8.0
38.0
18.0
34.0
15.0
16.0
17.0
18.0
28.0
45.0
23.0
16.0
24.0
12.0
25.0
73.0
35.0
114.0
15.0
23.0
16.0
1

In [ ]:
normalized_dtw = normalized_drw(dtw_distance)

In [ ]:
db_score = []
for num_c in cluster_num:
    db_score.append(normalized_dtw, num_c)

In [ ]:
#check the daily pattern for each individual 
arr_test = data2arr(raw_dt_path[0], to2d=False)


db_scroe_test = []
for i in arr_test.shape[0]:
    dtw_mx_, normalized_dtw_ = get_dtw_dist(arr_test[i], True)
    for num_c in cluster_num:
        db_score_test.append(normalized_dtw_, num_c)
    plt.plot(db_score_test)
    plt.title(f'individual idx = {i}')
    


In [21]:

db_index = davies_bouldin_score(normalized_dtw_distances, cluster_labels)

print("Davies-Bouldin Index:", db_index)

Davies-Bouldin Index: 2.53114765553973


In [7]:
arr.shape

(13709, 167, 288)

In [8]:
arr_reshaped = arr.reshape(arr.shape[0], -1)

In [12]:
#arr_reshaped = arr.reshape(arr.shape[0], -1)

# Convert the binary data into a document-term matrix
documents = [list(map(str, row)) for row in arr_reshaped]

# Create a dictionary and a corpus for Gensim
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Apply LDA-like topic modeling
lda_model = gensim.models.LdaModel(corpus, num_topics=5, id2word=dictionary, passes=15)
topics = lda_model.print_topics(num_words=10)

# Print the discovered topics
for topic in topics:
    print(topic)

(0, '1.000*"0.0" + 0.000*"1.0"')
(1, '0.999*"0.0" + 0.001*"1.0"')
(2, '1.000*"0.0" + 0.000*"1.0"')
(3, '1.000*"0.0" + 0.000*"1.0"')
(4, '1.000*"0.0" + 0.000*"1.0"')


starttime = time.time()

rank_individuals = 10
rank_locations = 10
rank_times = 12

core, factors = tucker(arr, rank=[rank_individuals, rank_locations, rank_times])
endtime = time.time()
print(f'running time {endtime-starttime}')

reconstructed_data = tl.tucker_to_tensor((core, factors))